In [12]:
import pandas as pd
from pykrx import stock
import matplotlib.pyplot as plt

In [13]:
df_kosdaq_150 = stock.get_market_ohlcv_by_date('20101001', '20241230', '229200')

In [14]:
df_kosdaq_150['변동폭'] = df_kosdaq_150['고가'] - df_kosdaq_150['저가']
df_kosdaq_150.head(4)

,시가,고가,저가,종가,거래량,등락률,변동폭
날짜,,,,,,,
2015-10-01,10161,10161,10094,10101,31634,NaN,67
2015-10-02,10135,10159,10135,10148,63224,0.465300,24
2015-10-05,10255,10265,10210,10231,43154,0.817895,55
2015-10-06,10248,10274,10172,10176,42399,-0.537582,102


In [15]:
df_kosdaq_150['전일변동폭'] = df_kosdaq_150['변동폭'].shift(periods=1)
df_kosdaq_150.head(4)

,시가,고가,저가,종가,거래량,등락률,변동폭,전일변동폭
날짜,,,,,,,,
2015-10-01,10161,10161,10094,10101,31634,NaN,67,NaN
2015-10-02,10135,10159,10135,10148,63224,0.465300,24,67.0
2015-10-05,10255,10265,10210,10231,43154,0.817895,55,24.0
2015-10-06,10248,10274,10172,10176,42399,-0.537582,102,55.0


In [17]:
 # 매수 조건: 고가가 목표가 이상일 경우
df_kosdaq_150['매수'] = df_kosdaq_150['고가'] >= df_kosdaq_150['목표가']

In [21]:
import numpy as np

# K값 범위 설정
k_values = np.arange(0.1, 1.1, 0.1)
results = []

for k in k_values:
    df["목표가"] = df["시가"] + (df["변동폭"] * k)
    df["매수"] = df["고가"] >= df["목표가"]  # 매수 조건
    df["수익률"] = np.where(df["매수"], (df["종가"] - df["목표가"]) / df["목표가"], 0)
    
    # 누적 수익률 계산
    cumulative_return = (1 + df["수익률"]).cumprod().iloc[-1] - 1
    results.append({"K값": k, "누적 수익률": cumulative_return})

In [22]:

#
# 결과 정렬 및 최적 K값 선택
result_df = pd.DataFrame(results).sort_values(by="누적 수익률", ascending=False)
optimal_k = result_df.iloc[0]

# 단순 보유 전략 계산
df["단순 수익률"] = (df["종가"] - df["시가"]) / df["시가"]
simple_cumulative_return = (1 + df["단순 수익률"]).cumprod().iloc[-1] - 1

# 결과 출력
print("최적 K값:", optimal_k["K값"])
print("최적 K값의 누적 수익률:", optimal_k["누적 수익률"])
print("단순 보유 전략의 누적 수익률:", simple_cumulative_return)

# 낮은 K값 필터링
low_k_values = result_df[result_df["누적 수익률"] < simple_cumulative_return]
print("단순 보유보다 낮은 K값들:\n", low_k_values)


최적 K값: 0.1
최적 K값의 누적 수익률: 0.2431617086881417
단순 보유 전략의 누적 수익률: 0.35971321882001495
단순 보유보다 낮은 K값들:
     K값    누적 수익률
0  0.1  0.243162
1  0.2  0.159976
2  0.3  0.082947
3  0.4  0.032572
7  0.8 -0.012214
6  0.7 -0.017832
8  0.9 -0.023367
4  0.5 -0.027834
9  1.0 -0.034333
5  0.6 -0.046459


NameError: name 'optimal_k' is not defined